In [25]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [26]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [27]:
funcap_proyectos = parser('http://montenegro.funcap.ce.gov.br/sugba/editais/')

#Abiertos

################################################
#Titulos

list_titulos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][1]//tr[1]//text()')
titulos_abiertos = []
for tit in list_titulos:
    tit = tit.strip()
    if tit != '':
        titulos_abiertos.append(tit)

################################################
#anos

anos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][1]//tr//td[2]/text()')
years_abiertos= []
for ano in anos:
    if ano == '\r\n                                    ':
        index = anos.index(ano)+1
        years_abiertos.append(anos[index].split('/')[2].strip())

################################################
#links

pdfs_proyectos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][1]//tr/td/a/@href')

corte = []


for pdf in pdfs_proyectos:
    if pdf.startswith('http://'):
        corte.append(pdfs_proyectos.index(pdf))

alist_pdfs = []

for i in range(len(corte)):
    try:
        if corte[i] != corte[-1]:
            alist_pdfs.append(pdfs_proyectos[corte[i]:corte[i+1]])
        else:
            alist_pdfs.append(pdfs_proyectos[corte[i]::])
    except:
        continue

pdfs_proyectos_abiertos = []
base = 'http://montenegro.funcap.ce.gov.br/sugba'

for pdfs in alist_pdfs:
    link = ''
    for pdf in pdfs:
        if pdf.startswith('http'):
            continue
        else:
            pdf = base + pdf.strip('..')
            link = link + pdf + ', '

    pdfs_proyectos_abiertos.append(link.strip(', '))

pdfs_proyectos_abiertos


['http://montenegro.funcap.ce.gov.br/sugba/edital/434.pdf, http://montenegro.funcap.ce.gov.br/sugba/edital/430.pdf, http://montenegro.funcap.ce.gov.br/sugba/edital/426.pdf',
 'http://montenegro.funcap.ce.gov.br/sugba/edital/432.pdf']

In [28]:

#Cerrados

###################################
#Titulos

list_titulos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][2]//tr[1]//text()')
titulos_cerrados = []
for tit in list_titulos:
    tit = tit.strip()
    if tit != '':
        titulos_cerrados.append(tit)

####################################
#anos

anos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][2]//tr//td[2]/text()')


n = 0
alist_anos = []
nums = []
for ano in anos:
    ano = ano.strip()
    if len(ano) == 0:
        alist_anos.append(str(n))
        nums.append(str(n))
        n+=1
    else:
        alist_anos.append(ano)

# ------------------------------------

years_cerrados = []
n = 0

for ano in alist_anos:
    
    if str(n) == ano:
        index = alist_anos.index(ano)+1        
        
        if str(n) != alist_anos[index]:
            
            years_cerrados.append(alist_anos[index].split('/')[-1])  
        
        n+=1

##########################
"""
Tener cuidado con los pdf, por falta de estrucutra en la pagina se imposibilita delimitarlos bien por proyecto, toca verificar de forma manual.
"""
#PDFs
pdfs_proyectos = funcap_proyectos.xpath('//div[@class="box ui-tabs ui-widget ui-widget-content ui-corner-all"][2]//tr/td/a/@href')


pdfs_proyectos2 = []
n = 0
kk = []
for pdf in pdfs_proyectos:
    
    if 'resultados' not in pdf:
        kk.append(pdf)
    else:
        if len(kk) == 0:
            continue
        pdfs_proyectos2.append(kk)
        kk=[]
        

pdfs_proyectos_cerrados= []
base = 'http://montenegro.funcap.ce.gov.br/sugba'

for pdfs in pdfs_proyectos2:
    link = ''
    for pdf in pdfs:
        pdf = base + pdf.strip('..')
        link = link + pdf + ', '

    pdfs_proyectos_cerrados.append(link.strip(', '))

len(pdfs_proyectos_cerrados)



107

In [29]:

# Pdfs
pdfs_proyectos_abiertos+=pdfs_proyectos_cerrados

#anos
years_abiertos+=years_cerrados

#titulo
titulos_abiertos+=titulos_cerrados



In [30]:
for i in range(22):

    pdfs_proyectos_abiertos.append('')

len(pdfs_proyectos_abiertos)

131

In [31]:
funcap = pd.DataFrame()

funcap['Titulo'] = titulos_abiertos
funcap['Ano'] = years_abiertos
funcap['pdfs'] = pdfs_proyectos_abiertos

funcap.to_excel('brasil_funcap.xlsx')